# Acoustics equations and their Riemann solver in 1D and 2D
by <a href="mailto:maojrs@uw.edu">Mauricio J. Del Razo S.</a> 2014

A brief intuitive derivation of the acoustics equations is presented. We derive the normal Riemann solver for one and two dimensional acoustics and the transverse Riemann solver for two dimensional acoustics in a cartesian grid. These solvers are further extended for general quadrilateral mapped grids. Along some of these examples, we will show how to use PyClaw to solve the acoustics equations.

## 1. Linear Acoustics equations
The linear acoustic equations can be obtained by linearizing the conservation of mass and momentum for an element of fluid. The conservation of mass and momentum are given by

$$
\frac{\partial \rho}{\partial t} + \nabla \cdot \left(\rho \bar{u}\right) = 0 \\
\frac{d \rho \bar{u}}{dt} = -\nabla P
$$

where $d/dt$ denotes the material derivative: $\frac{d}{dt} = \frac{\partial}{\partial t} + \bar{u} \cdot \nabla $. In one dimension, these can be easily rewritten as,

$$
\rho_t + (\rho u)_x = 0, \\
(\rho u)_t + (\rho u^2 + P(\rho))_x =0,
$$

where we assumed the pressure is a function of the density. Linearizing the equation around $\rho_0$ and $u_0 =0$, we obtain the simpler system for the perturbations,

$$
\rho_t + (\rho u)_x = 0, \\
(\rho u)_t + P(\rho_0)\rho_x =0.
$$

where $\rho$ and $u$ are now the perturbations around $\rho_0$ and $u_0 = 0$. As perturbations on the pressure and density satisfy, 

$$
p\approx P'(\rho_0)\rho, \\
\rho u \approx \rho_0 u,
$$

we can rewrite the system of linear acoustic equations as

$$
 \left[ \begin{array}{c}
p \\
u 
\end{array} \right]_t
+  \underbrace{\left[ \begin{array}{cc}
0 & K_0 \\
1/\rho_0 & 0  \\
\end{array} \right]}_{\mathbf{A}}
\left[ \begin{array}{c}
p \\
u \end{array} \right]_x = 0,
$$

with $K_0=\rho_0 P'(\rho_0)$ the bulk modulus of compressibility that will tell us how compressible is a material. This can be easily extended to two dimensions,

$$
 \left[ \begin{array}{c}
p \\
u \\
v
\end{array} \right]_t
+ \underbrace{\left[ \begin{array}{ccc}
0 & K_0 & 0 \\
1/\rho_0 & 0 & 0   \\
0 & 0 & 0 \\
\end{array} \right]}_{\mathbf{A}}
\left[ \begin{array}{c}
p \\
u \\
v \end{array} \right]_x
+\underbrace{\left[ \begin{array}{ccc}
0 & 0 & K_0  \\
0 & 0 & 0   \\
1/\rho_0 & 0 & 0 \\
\end{array} \right]}_{\mathbf{B}}
\left[ \begin{array}{c}
p \\
u \\
v \end{array} \right]_y = 0
$$

and three dimensions


$$
 \left[ \begin{array}{c}
p \\
u \\
v \\
w
\end{array} \right]_t
+ \underbrace{\left[ \begin{array}{ccc}
0 & K_0 & 0 & 0 \\
1/\rho_0 & 0 & 0 & 0   \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{array} \right]}_{\mathbf{A}}
\left[ \begin{array}{c}
p \\
u \\
v \\
w\end{array} \right]_x
+\underbrace{\left[ \begin{array}{ccc}
0 & 0 & K_0 & 0  \\
0 & 0 & 0 & 0  \\
1/\rho_0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0\\
\end{array} \right]}_{\mathbf{B}}
\left[ \begin{array}{c}
p \\
u \\
v \\
w\end{array} \right]_y 
+\underbrace{\left[ \begin{array}{ccc}
0 & 0 & 0 & K_0  \\
0 & 0 & 0 & 0  \\
0 & 0 & 0 & 0 \\
1/\rho_0 & 0 & 0 & 0\\
\end{array} \right]}_{\mathbf{C}}
\left[ \begin{array}{c}
p \\
u \\
v \\
w\end{array} \right]_z= 0
$$
where the velocity in $x,y,z$ is $u,v,w$ and the matrices $A$, $B$ and $C$ have those forms depending on the dimensionality.

## 2. The Riemann problem in 1D (extended to discontinuous density and bulk modulus)
In one dimension we need to solve,

$$
\bar{q}_t + \mathbf{A} \bar{q}_x = 0, \\
$$
with
$$
\bar{q} = \left[ \begin{array}{c}
p \\
u
\end{array}\right]
\hspace{20mm}
\mathbf{A} = 
\left[ \begin{array}{cc}
0        & K_0 \\
1/\rho_0 & 0
\end{array} \right],
$$
and initial condition
$$
\bar{q}(x,0) = \begin{cases}
\bar{q_L} & \text{if   } x \le 0, \\
\bar{q_R} & \text{if   } x > 0.
\end{cases}
$$
The solution of the Riemann problem can be obtained by transforming the system into two uncoupled advection equations, see [LeVeque 2002](http://depts.washington.edu/clawpack/book.html). When transformed back to the original coordinates the solution will have the following structure, 

<img style="float: center" src="images/acoustics_RP.png" width="300">

Two propagating acoustic waves (one to the left and one to the right) with speeds $c_L$ and $c_R$, the two initial conditions $q_L$ and $q_R$ in the far field and one more extra state in between the waves $q_M$. If we can find $c_L$, $c_R$ and $q_M$, we have solved the problem.

The following section show how to obtain this solution to the Riemann problem, for details on the methodology, see [LeVeque 2002](http://depts.washington.edu/clawpack/book.html). We will begin by obtaining the eigenvector and eigenvalues of $\mathbf{A}$ symbolically (although this might be very simple to do by hand, on more complicated systems of equations might become handy).

In [ ]:
# Initialize sympy for symbolic computations
import sympy as sy
# Mathjax printing with simpy
sy.init_printing(use_latex='mathjax')

# Declare symbolic variables
rho_0, K_0 = sy.symbols('rho_0 K_0', real=True)
R = sy.Matrix(2, 2, lambda i,j: 0)
Rval = sy.Matrix(2, 1, lambda i,j: 0)

# Define symbolic matrix
An = sy.Matrix([[0,     K_0],
                 [1/rho_0, 0]])

# Calculate eigenvalues and eigenvectors of matrix An
# eigvec[i][0] => ith eigenvalue
# eigvec[i][2] => ith eigenvector
eig_vec = An.eigenvects()


# Simplify eigenvalues and eigenvectors and save in symbolic arrays
for i in range(2):
    Rval[i] = sy.simplify(eig_vec[i][0])
    for j in range(2):
        R[j,i] = sy.simplify(eig_vec[i][2][0][j])

In [ ]:
# Show eigenvalue and corresponding eigenvectors in a widget

#from IPython.html.widgets import interact, interactive
from IPython.display import display, HTML, Latex
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

# Function to print the eigenvalue and eigenvector
def print_eigen(Eigenvalue):
    display(Latex('Eigenvalue =' + '$' + sy.latex(Rval[Eigenvalue]) + '\hspace{20mm} $' + 
                  'Eigenvector =$' + sy.latex(R[:,Eigenvalue]) + '$'))
    
# Interactive widget to display eigenvalue and eigenvector
interact(print_eigen, Eigenvalue = {'one':0,'two':1});

The eigenvalues and eigenvectors obtained are the same than those in [LeVeque 2002](http://depts.washington.edu/clawpack/book.html), except for constant factors in the eigenvectors. The speeds of the left and right going acoustic waves are given by the eigenvalues: $s_L = -c_0$, $s_R = c_0$ with:  $c_0 = \sqrt{K_0/\rho_0}$.
However, we would like to have different materials in the left and right side, which means different density ($\rho_L$ and $\rho_R$) and bulk modulus ($K_L$ and $K_R$), therefore
$$
s_L = -c_L \hspace{20mm} s_R = c_R \hspace{20mm} \mathrm{with:} \hspace{5mm} c_M = \sqrt{\frac{K_{M}}{\rho_{M}}},
$$
with $M=L,R$. Using the fact that $K_{M}=c_{M}^2 \rho_{M}$ and recalling the specific acoustic impedance is given by $Z_{M}=\rho_{M} c_{M} = K_M/\sqrt{K_M/\rho_M}$, we can write the matrix of column eigenvectors $\mathbf{R}=[\bar{r_L}, \bar{r_R}]$ as,
$$
\mathbf{R} = 
\left[ \begin{array}{ccccc}
-Z_{L} & Z_{R} \\
 1   & 1  \\
\end{array} \right],
$$

Let  $\delta \bar{Q} = [\Delta_p, \Delta_u]^T$  be the jump of $\bar{q}$ across the discontinuity. In order to solve the Riemann problem, we need to expand the jump accross the discontinuity $\Delta \bar{q}$ as a linear combination of the eigenvectors,
$$
\alpha_L \bar{r_L} + \alpha_R \bar{r_R} = \Delta \bar{q},
$$
or more compactly
$$
\mathbf{R} \bar{\alpha} = \Delta \bar{q}.
$$
We will need to obtain the value of $\bar{\alpha}$, solving again this equation symbolically for illustrative purposes,

In [ ]:
# Declare symbolic variables
ZL, ZR, dp, du = sy.symbols('Z_L Z_R Delta_p Delta_u', real=True)
R = sy.Matrix(2, 2, lambda i,j: 0)

# Define symbolic matrix and vector
R = sy.Matrix([[-ZL, ZR],
               [1,   1]])
dq = sy.Matrix([[dp],[du]])

# Solve system
alpha = R.inv()*dq

#Simplify and print
al = sy.simplify(alpha[0])
ar = sy.simplify(alpha[1])
display(Latex('$\\alpha_L =$' + '$' + sy.latex(al) + '$' ))
display(Latex('$\\alpha_R =$' + '$' + sy.latex(ar) + '$' ))


We already know the wave speeds $c_L$ and $c_R$ and the initial states $q_L$ and $q_R$. We only need to know the middle state $q_M$, which will be given by
$$
\bar{q_M} = \bar{q_L} + \alpha_L \bar{r_L} = \bar{q_R} - \alpha_R \bar{r_R}.
$$

### 2.1 Implementing a sample solution into python
Now we can plot and construct the solution to the Riemann problem. We will give an outline of the algorithm followed by the code:
<ol>
<li> Define piecewise constant initial condition with the left and right states and corresponding bulk modulii,
$$
q_L = [p_L, u_L] \ \ \text{with} \ \ K_L, \\
q_R = [p_R, u_R] \ \ \text{with} \ \ K_R. 
$$
</li>
<li> Calculate the wave speeds given by the eigenvalues and the corresponding eigenvectors of $\mathbf{A}$.
<li> Compute $\bar{\alpha}$ that are the weights of the jumps in the direction of each of the eigenvectors </li>
<li> Compute the middle state $q_{m}$ using the following formulas for the middle states,

$$
q_{m} = q_{L} + \alpha_L \bar{r_L}  \ \ \ \mathrm{or} \\
q_{m} = q_{R} - \alpha_R \bar{r_R}
$$
</li>
Now, we will implement this into a python script and plot the solution.

In [ ]:
# SOLUTION IMPLEMENTATION
#import numpy
import numpy as np

# Set left and right state and deltaq (q = [sig11,sig22,sig12,u,v])
qL = np.array([-1.0, 0.0])
qR = np.array([1.0, 0.0])
dq = np.array([qR[0]-qL[0], qR[1]-qL[1]])

# Set bulk and density (left and right)
bulkL = 1;   rhoL = 1 
bulkR = 4;   rhoR = 2

# Define speeds and impedance (left and right)
cL = np.sqrt(bulkL/rhoL);  ZL = rhoL*cL
cR = np.sqrt(bulkR/rhoR);  ZR = rhoR*cR

# Define the 2 eigenvectors (from columns of Matrix R)
r1 = np.array([-ZL, 1])
r2 = np.array([ZR,  1])

# Compute the 2 alphas
det = ZL + ZR
alL = (-dq[0] + dq[1]*ZR)/det
alR = (dq[0] + dq[1]*ZL)/det

# Compute middle state qm
qm = qL + alL*r1 
## Should be equivalent to
#qms = qR - alR*r2 #it is!
    
# Compute waves characteristics for plotting
x = np.linspace(-5,5,50)
Wc = np.zeros((2,len(x)))
Wc[0][:] = -x/cL
Wc[1][:] = x/cR

In [ ]:
#SOLUTION PLOTTING
# Required for plotting
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
#from IPython.html.widgets import interact
#from ipywidgets import StaticInteract, RangeWidget, RadioWidget, DropDownWidget

# Plot Riemann solution
def plot_Riemann(time):
    
    fig = plt.figure(figsize=(15, 5))
    
    # Create subplot for (x,t) plane
    tmax = 5
    ax = fig.add_subplot(1,3,1)
    ax.set_xlabel('x')
    ax.set_ylabel('time')
    ax.axis([min(x),max(x),0,tmax])
    
    # Plot characteristic lines
    # Acoustic waves in red
    ax.plot(x,Wc[0][:], '-r', linewidth=2)
    ax.plot(x,Wc[1][:], '-r', linewidth=2)
    # Plot time-line in (x,t) plane
    ax.plot(x, 0*x+time, 'k', linewidth=3)

    # Create pressure subplot for Riemann solution
    ax2 = fig.add_subplot(1,3,2)
    ax2.set_xlabel('x')
    ax2.set_ylabel('Pressure')
    ax2.axis([min(x),max(x),-2,2])
    
    # Create Riemann solution vector and plot
    sol = np.zeros(len(x))
    for i in range(len(x)):
        if x[i] < -cL*time:
            sol[i] = qL[0]
        elif x[i] < cR*time:
            sol[i] = qm[0]
        else:
            sol[i] = qR[0]
    ax2.plot(x,sol, 'k', linewidth = 3)
    ax2.fill_between(x,-20, sol, facecolor='blue', alpha=0.2)
    
    # Create velocity subplot for Riemann solution
    ax3 = fig.add_subplot(1,3,3)
    ax3.set_xlabel('x')
    ax3.set_ylabel('Velocity')
    ax3.axis([min(x),max(x),-2,2])
    
    # Create Riemann solution vector and plot
    sol = np.zeros(len(x))
    for i in range(len(x)):
        if x[i] < -cL*time:
            sol[i] = qL[1]
        elif x[i] < cR*time:
            sol[i] = qm[1]
        else:
            sol[i] = qR[1]
    ax3.plot(x,sol, 'k', linewidth = 3)
    ax3.fill_between(x,-20, sol, facecolor='blue', alpha=0.2)
    return     
# Create interactive widget to visualize solution 
#interact(plot_Riemann, time=(0,5,0.1), qvar={'sigma11':0, 'sigma22':1, 'sigma12':2, 
                                             #'normal velocity u':3, 'transverse velocity v':4});
interact(plot_Riemann, time=(0,5,0.25)) 

The solution shows the $x-t$ to show the charactersitics and a horizontal line to show the current time. The other two plots show the solution for the pressure $p$ and velocity $u$ as a function of $x$, and the time slider lets you adjust
the time to observe the time evolution of the solution.

One way to understand how the middle state $q_M$ is chosen is to observe the phase plane. For instance, in the plot below we show the phase plane where the pressure $p$ corresponds to the $x$ axis and the velocity $u$ corresponds to the $y$ axis. In this plot, the initial left and right states $q_L$ and $q_M$ are points in the plane as well as the new middle state $q_M$. We show the phase plane for some specific parameters. The eigenvector directions are shown in the bottom of the interactive plot. The location of the middle state in the phase plane is obtain by the intersection of the parallelogram created by the eigenvectors and the initial states, which of the two intersections will depend on the location of the initial states. The solution will always move from $q_L$ to $q_M$ in the direction of its corresponding eigenvector $\bar{r_1}$ and the to $q_R$ in the direction of $\bar{r_2}$. This is well illustarted in the following interactive app. 

In [ ]:
from IPython.display import HTML
HTML('<iframe src=apps/phase_plane_acoustics_small.html width=900 height=400 scrolling=no frameborder=0 ></iframe>')

## 3. The normal Riemann problem in 2D
When solving a 2D problem, we can use dimensional splitting or unsplit methods. In any of those cases, we will need to solve a Riemann solver in the normal direction. In the $x$ direction, this will involve solving $\bar{q}_t + \mathbf{A}\bar{q}_x=0$ and in the $y$ direction it will solve $\bar{q}_t + \mathbf{B}\bar{q}_y=0$; we will only solve it in the $x$ direction, since it's completely analogous to solve it for the $y$ direction. The procedure to solve this problem will be the same than for the one-dimensional Riemann problem already explained in detail, so we will present the solution very briefly. We will start with the $\mathbf{A}$ matrix,

$$
\mathbf{A} =\left[ \begin{array}{ccc}
0 & K_0 & 0 \\
1/\rho_0 & 0 & 0   \\
0 & 0 & 0 \\
\end{array} \right].
$$

The matrix of column eigenvectors $\mathbf{R_x}=[\bar{r_L}, \bar{r_0}, \bar{r_R}]$ is

$$
\mathbf{R_x} = 
\left[ \begin{array}{ccc}
-Z_{0} & 0 & Z_{0} \\
 1   & 0 & 1  \\
 0 & 1 & 0
\end{array} \right],
$$

with $Z_0=\rho_0 c_0$ and $c_0=\sqrt{K_0/\rho_0}$.
Solving the system $\mathbf{R}\bar{\alpha}=\delta \bar{q} =[\Delta_p, \Delta_u, \Delta_v]^T$, we obtain

$$
\alpha_L = \frac{-\Delta_p + Z_0 \Delta_u}{2Z_0},\\
\alpha_0 = \Delta_v,\\
\alpha_R = \frac{\Delta_p + Z_0 \Delta_u}{2Z_0},
$$

and the speed $s_L = -c_0$, $s_0=0$ and $s_R = c_0$ correspond to the eigenvalues of the matrix $\mathbf{A}$. Note in this case we assumed the same material on both sides. With this information, we can reconstruct the solution to the Riemann problem as before.

### 3.1 Clawpack implementation

Clawpack has all the framework required to solve hyperbolic equations by solving Riemann problems on each grid cell. If we can provide the Riemann solver to Clawpack, it can do all the hard work for us. In order to implement the solution into Clawpack, we need to rewrite it in its wave formulation, where we update the value at cell $\bar{Q}_{i,j}^n$, with $i,j$ the indexes in space and $n$ in time following,

$$
\bar{Q}_{i,j}^{n+1} = \bar{Q}_{i,j}^{n} - \frac{\Delta t}{\Delta x}
\left[A^+\Delta Q_{i-1/2,j} + A^-\Delta Q_{i+1/2,j}\right] + \text{High order corrections & limiters},
$$ 

when sweeping in the $i$ direction (normal direction) of the computational domain. The same process is analogous in the $j$ direction. Also note this equation requires output of two Riemann problems: the one in edge $i-1/2$ and the one in $i+1/2$. The output of the Riemann solver routine for Clawpack should be given by the waves speed and wave fluctuactions. The wave speed we already know from the eigenvalues in the previous section. In order to calculate the wave fluctuations $A^{\pm}\Delta Q_{i-1/2,j}$ from the left edge, we first need the waves given by

$$
W_i = \alpha_i \bar{r_i}
$$

with $i=L,0,R$ and $\bar{r_i}$ the corresponding eigenvectors calculated previously. Solving the Riemann problem in the right edge is completely analogous. The positive and negative wave fluctuations $A^\pm \Delta Q_{i- 1/2,j}$ will be given in terms of the speeds(eigenvalues) and the waves,

$$
A^+ \Delta Q_{i-1/2,j} = s_R W_R \\
A^- \Delta Q_{i-1/2,j} = s_L W_L,
$$
Note the wave $W_0$ doesn't really make any difference in the fluctuations since it has speed zero.

## 4. The transverse Riemann problem in 2D

In order to obtain second order accuracy using unsplit methods, we will need to solve a transverse Riemann solver in addition to the normal one. This will require to split the normal wave fluctuactions $A^\pm \Delta Q_{i- 1/2,j}$ at edge $i-1/2$ into transverse wave fluctuactions $B^\pm A^+\Delta Q_{i- 1/2,j}$ and $B^\pm A^-\Delta Q_{i- 1/2,j}$; the former descomposition is shown in the following figure and explained in detail on [LeVeque 2002](http://depts.washington.edu/clawpack/book.html).
<img style="float: center" src="images/transverse_diag.png", width=500>

We will only focuse on $B^\pm A^+\Delta Q_{i- 1/2,j}$, since the other case is analogous. We begin by decomposing the normal wave fluctuations into a linear combination of transverse waves, which will follow the same equations and jacobian as before, so we obtain

$$
A^+\Delta Q_{i- 1/2,j} = \mathbf{R_y} \bar{\beta} = \beta_D \bar{r_D} + \beta_M \bar{r_M} + \beta_U \bar{r_U},
$$

where the subindexes $B,M$ and $U$ denote bottom, middle and upper direction. Where $\bar{r_i}$ are the column eigenvectors of $\mathbf{B}$, if sweeping in the $x$ direction or of $\mathbf{A}$ if sweeping on the $y$ direction. In this case, we will only consider the case when sweeping in the $x$ direction, since the other is analogous, so the matrix of eigenvectors of $\mathbf{B}$ is now $\mathbf{R_y}=[\bar{r_B},\bar{r_M},\bar{r_U}]$, i.e.

$$
\mathbf{R_y} = 
\left[ \begin{array}{ccc}
-Z_{0} & 0 & Z_{0} \\
 0   & 1 & 0  \\
 1 & 0 & 1
\end{array} \right].
$$

We solve the system in the same way than the system for $\bar{\alpha}$. When we add the right input parameters for the eigenvectors, we obtain $\bar{\beta}$ is given by,

$$
\beta_B = \frac{-A^+_1 + z_0 A^+_3}{2 Z_0}, \\
\beta_M = A^+_2, \\
\beta_U = \frac{A^+_1 + z_0 A^+_3}{2 Z_0}, \\
$$

where the fluctuation vector $A^+\Delta Q_{i- 1/2,j}=[A^+_1, A^+_2,A^+_3]$, where the speeds 

The speeds (eigenvalues) are 
$$
s_B = -c_{0} \ \ s_M = 0 \ \ s_U = c_{0},
$$

with $c_0=\sqrt{K_0/\rho_0}$. The transverse waves are given by $W_i = \beta_i \bar{r_i}$ with $i=B,M,U$, so the fluctuations that will be required to be implemented into Clawpack are given by,

$$
B^+A^+ \Delta Q_{i-1/2,j} = s_U W_U \\
B^-A^+ \Delta Q_{i-1/2,j} = s_B W_B.
$$

Note once again the middle wave has speed zero, so it doesn't affect these fluctuations. 

Now we will show how the normal and transverse flucutuations propagate from the grid edges to the grid cells. We first
obtain the solution for the normal and transverse fluctuation and then we plot them. Note this plot doesn't really show the difference in the vlaues for the pressure and velocities, but it does illustrate the wave speeds and how the normal and transvere fluctuations propagate.

In [ ]:
# NORMAL AND TRANSVERSE RIEMANN SOLVER SOLUTION (cartesian grid)
#import numpy
import numpy as np

# Set left and right state and deltaq (q = [sig11,sig22,sig12,u,v])
qL = np.array([1.0, 0.0, 0.0])
qR = np.array([0.0, 0.0, 0.0])
dq = np.array([qR[0]-qL[0], qR[1]-qL[1],qR[2]-qL[2]])

# Set normal for normal solver
nx = 1.0
ny = 0.0
# Set normal for transverse solver
nxt = 0.0
nyt = 1.0

# Set grid size (dx2=dx/2) and time step
dx2 = 0.01
dy2 = 0.01
dt = 0.01

# Set bulk and density (left and right)
bulkL = 1;   rhoL = 1 
bulkR = 8;   rhoR = 2

# Define speeds and impedance (left and right)
cL = np.sqrt(bulkL/rhoL);  ZL = rhoL*cL
cR = np.sqrt(bulkR/rhoR);  ZR = rhoR*cR

## NORMAL SOLVER

# Define the 2 eigenvectors (from columns of Matrix R)
rL = np.array([-ZL, nx, ny])
rR = np.array([ZR, nx, ny])
# Define eigenvalues
sL = -cL
sR = cR

# Compute the 2 alphas
det = ZL + ZR
alL = (-dq[0] + (nx*dq[1] + ny*dq[2])*ZR)/det
alR = (dq[0] +  (nx*dq[1] + ny*dq[2])*ZL)/det

# Compute wave fluctuations
amdq = alL*rL*sL
apdq = alR*rR*sR

## TRANSVERSE SOLVER (assuming same material on top and bottom cells)
# Define the 2 eigenvectors (from columns of Matrix R)
rB = np.array([-ZR, nxt, nyt])
rU = np.array([ZR, nxt, nyt])
# Define eigenvalues
sB = -cR
sU = cR

det = 2.0*ZR
beB = (-apdq[0] + (nxt*apdq[1] + nyt*apdq[2])*ZR)/det
beU = (apdq[0] +  (nxt*apdq[1] + nyt*apdq[2])*ZR)/det

# Compute transverse wave fluctuations
bmdq = beB*rB*sB
bpdq = beU*rU*sU

In [ ]:
# CREATE INTERACTIVE PLOT
# Required for plotting
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.patches import Ellipse, Polygon
from pylab import *
#from IPython.html.widgets import interact
#from ipywidgets import StaticInteract, RangeWidget, RadioWidget, DropDownWidget

# Plot diagram for transverse solver
 
def plot_trans_diag(dt,up_wave,dwn_wave):
    x = np.linspace(-2*dx2,2*dx2,50);
    y1 = 0.0*x + dy2;
    y2 = 0.0*x - dy2;

    y = np.linspace(-2*dy2,2*dy2,50);
    x1 = 0.0*y + dx2;
    x2 = 0.0*y - dx2;
 
    fig = plt.figure()
    ax = fig.add_subplot(111,aspect='equal') 
    
    ax.plot(x,y1,'-k',linewidth=2)
    ax.plot(x,y2,'-k',linewidth=2)
    ax.plot(x1,y,'-k',linewidth=2)
    ax.plot(x2,y,'-k',linewidth=2)
    ax.axis([min(x),max(x),min(y),max(y)])
    ax.axis('off')
    
    xplus = sR*dt -dx2
    xminus = sL*dt -dx2
    yplus = sU*dt
    yminus = sB*dt
    
    # Main patches (A+DQ and A-DQ)
    verts = [-dx2,-dy2], [xplus,-dy2], [xplus,dy2], [-dx2,dy2] 
    poly = patches.Polygon(verts, facecolor='blue', alpha=0.5, linewidth=3) 
    ax.add_patch(poly)
    
    verts = [-dx2,-dy2], [xminus,-dy2], [xminus,dy2], [-dx2,dy2] 
    poly = patches.Polygon(verts, facecolor='blue', alpha=0.5, linewidth=3) 
    ax.add_patch(poly)
    
    #Transverse patches
    if (up_wave):
        verts = [-dx2,-dy2], [xplus,-dy2+yplus], [xplus,dy2+yplus], [-dx2,dy2] 
        poly = patches.Polygon(verts, facecolor='yellow', alpha=0.5, linewidth=3) 
        ax.add_patch(poly) 
        
    if (dwn_wave):
        verts = [-dx2,-dy2], [xplus,-dy2+yminus], [xplus,dy2+yminus], [-dx2,dy2] 
        poly = patches.Polygon(verts, facecolor='red', alpha=0.5, linewidth=3) 
        ax.add_patch(poly) 
    
interact(plot_trans_diag, up_wave=True, dwn_wave=True, dt=(0.0,0.008,0.0004))

## 5. The normal and transverse Riemann problem for quadrilateral mapped grids (and varying density and bulk modulus)

In order to solve the normal and transverse acoustics Riemann problems for general quadrilateral mapped grids, we will need to solve them in the normal and transverse direction to a grid cell. We can do this by the same process as before, but using a matrix that represents the problem in the normal or transverse direction. The Riemann problem in the normal direction of a grid edge is $\bar{q}_t + \mathbf{A_n}\bar{q}_n=0$, where $\bar{q}_n$ is the derivative in the normal to the edge direction,  $\mathbf{A_n}=n_x \mathbf{A} +  n_y\mathbf{B}$, and $\hat{n}=(n_x,n_y)$ is the normal to the edge where the --normal or transverse-- Riemann problem is being solved. The jacobian matrix is,

$$
\mathbf{A_n} =
\left[ \begin{array}{ccccc}
0 & n_x K_0 & n_y K_0  \\
n_x/\rho_0 & 0 & 0  \\
n_y/\rho_0 & 0 & 0 \\
\end{array} \right],
$$

Using different materials on the left and right side, the matrix of eigenvectors $\mathbf{R}=[\bar{r_L},\bar{r_0},\bar{r_R}]$ is now

$$
\mathbf{R} = 
\left[ \begin{array}{ccc}
-Z_{L} & 0 & Z_{R} \\
 n_x   & -n_y & n_x  \\
 n_y & n_x & n_y
\end{array} \right],
$$

with $Z_i=\rho_i c_i$ and $c_i = \sqrt{K_i/\rho_i}$, where we now have two different values for the impedance, $i=L,R$,
on the left and right side indicating different materials. The respective eigenvalues are again given by $s_L=-c_L$, $s_0=0$ and $s_R=c_R$.In order to solve the Riemann problem, we need to solve again $\mathbf{R}\bar{\alpha} = \delta \bar{q}=[\Delta_p, \Delta_u, \Delta_v]^T$,

$$
\alpha_L = \frac{-\Delta_p + Z_R(n_x\Delta_u + n_y\Delta_v)}{Z_L+Z_R} \\
\alpha_0 = n_x\Delta_v - n_y \Delta_u \\
\alpha_R = \frac{\Delta_p + Z_L(n_x\Delta_u + n_y\Delta_v)}{Z_L+Z_R}
$$


The clawpack implementation will when employing the mapped grid we will need to scale the wave speeds $s$ in the wave fluctuations by the factor $\gamma_x = dy_{phy}/dy_{com}$ if sweeping in the $x$ direction or $\gamma_y = dx_{phy}/dx_{com}$ when sweeping in the $y$ direction, where $dx_{phy}$ denotes mesh grid spacing in the physical domain and $dx_{com}$ in the computational one. The fluctuation for the normal solver in the mapped grid should then be,

$$
A^+ \Delta Q_{i-1/2,j}^{map} = \gamma s_R \alpha_R \bar{r_R}, \\
A^- \Delta Q_{i-1/2,j}^{map} = \gamma s_L \alpha_L \bar{r_L},
$$

where $\gamma$ should be $\gamma_x$ or $\gamma_y$ depending on the sweeping direction.
Analogously, the up and bottom eigenvectors for the transverse solver will be in principle the same, but with their corresponding normals $\bar{r_B} = [Z_B,n{Bx},n_{By}]^T$, $\bar{r_U} = [Z_B,n{Ux},n_{Uy}]^T$, and eigenvalues $s_U=c_U$ and $s_B=-c_B$. Solving the usual system results in,

$$
\beta_B = \frac{-A^+_1 + Z_U(n_{Mx} A^+_2 + n_{My} A^+_3)}{Z_U+Z_B} \\
\beta_U = \frac{A^+_1 + Z_B(n_{Ux} A^+_2 + n_{Uy} A^+_3)}{Z_U+Z_B},
$$

where $A^+\Delta Q_{i- 1/2,j}=[A^+_1, A^+_2,A^+_3]$, $\hat{n_B}=(n_{Mx},n_{My})$ is the normal to the lower edge of
the middle cell and  $\hat{n_U}=(n_{Ux},n_{Uy})$ is the normal to lower edge of the upper cell.


and for the transverse solver,

$$
B^+A^+ \Delta Q_{i-1/2,j}^{map} = \gamma_U s_{U} \beta_U \bar{r_U} \\
B^-A^+ \Delta Q_{i-1/2,j}^{map} = -\gamma_M s_{B} \beta_B \bar{r_B},
$$

where $\gamma_U$ correspond to the scaling ratio of the bottom edge of the upper cell and $\gamma_M$ to the scaling ratio of the bottom edge of the middle cell. The same process can be repeated for the left normal fluctuation $A^-\Delta Q_{i-1/2,j}$. Note the direction of the normal solver is the direction in which we are sweeping, which could be $x$ or $y$ direction of our computational domain.
